In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/서울시 우리마을가게 상권분석서비스(상권-추정매출).csv', encoding='cp949')

In [3]:
df['서비스_업종_코드_명'].unique()

array(['전자상거래업', '인테리어', '가전제품', '가구', '화초', '섬유제품', '완구', '운동/경기용품',
       '화장품', '문구', '의약품', '시계및귀금속', '안경', '가방', '신발', '일반의류', '반찬가게',
       '핸드폰', '컴퓨터및주변장치판매', '편의점', '슈퍼마켓', '피부관리실', '네일숍', '미용실', '스포츠클럽',
       '치과의원', '일반의원', '커피-음료', '분식전문점', '패스트푸드점', '제과점', '양식음식점',
       '일식음식점', '중식음식점', '한식음식점', '의료기기', '노래방', '고시원', '여관', '세탁소',
       '자동차수리', '골프연습장', '당구장', '한의원', '스포츠 강습', '호프-간이주점', '치킨전문점',
       '조명용품', '철물점', '애완동물', '서적', '청과상', '수산물판매', '육류판매', '미곡판매',
       '가전제품수리', 'PC방', '예술학원', '외국어학원', '일반교습학원', '자동차미용', '부동산중개업',
       '자전거 및 기타운송장비'], dtype=object)

# 1. 데이터 셋 기준 요식업 정의

In [4]:
food = ['커피-음료', '분식전문점', '패스트푸드점', '제과점', '양식음식점','일식음식점', '중식음식점', '한식음식점', '호프-간이주점', '치킨전문점']

In [5]:
# df_food_1 = df[df['서비스_업종_코드_명'].isin(food)]

# 2. 데이터 내 seasonality 제거

 계절성을 없애고 기준년, 상권구분, 상권코드, 서비스 업종 기반으로 분석하기 위해 데이터 전처리 

'분기당_매출_금액' : 계절성을 없애기 위해 분기별 평균금액 사용  
'xxx건수' : 비율 확인을 위해 분기 합계 수치 계산 

In [6]:
df.shape

(140830, 80)

In [7]:
df_2 = df.groupby(['기준_년_코드','상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명']).mean()[['분기당_매출_금액']].reset_index()

In [8]:
df_2.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액
0,2021,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,6.897200e+08
1,2021,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,1.217034e+08
2,2021,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,9.493560e+06
3,2021,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,1.905703e+07
4,2021,A,골목상권,2110001,이북5도청사,CS200001,일반교습학원,4.620523e+07


In [9]:
groupby_col = ['분기당_매출_건수','주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',
       '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_건수~06_매출_건수',
       '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수',
       '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수', '여성_매출_건수',
       '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수',
       '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '점포수']

In [10]:
df_2[groupby_col] = df.groupby(['기준_년_코드','상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명']).sum()[groupby_col].reset_index().drop(columns = df.columns[[0,2,3,4,5,6,7]])

In [11]:
df_2.head()[['상권_코드','서비스_업종_코드', '분기당_매출_건수', '남성_매출_건수', '여성_매출_건수']]

,상권_코드,서비스_업종_코드,분기당_매출_건수,남성_매출_건수,여성_매출_건수
0,2110001,CS100001,113863,68698,33363
1,2110001,CS100008,52280,29790,18303
2,2110001,CS100009,1221,758,341
3,2110001,CS100010,6307,2145,3816
4,2110001,CS200001,552,296,256


In [12]:
# 원본데이터도 잘 맞지 않음...
# df.loc[(df['상권_코드']==1001496)&(df['서비스_업종_코드']=='CS300043'), ['상권_코드','서비스_업종_코드', '분기당_매출_건수', '남성_매출_건수', '여성_매출_건수']]


# 3. 파생변수 생성

. 아래 매출건수를 비율 변환  (ex > 남성매출건수/분기당매출건수)  
    ['분기당_매출_건수','남성_매출_건수', '여성_매출_건수',
    '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수',
     '연령대_50_매출_건수', '연령대_60_이상_매출_건수']


In [13]:
# df_food2 = df_food_1.copy()
df_3 = df_2.copy()

In [14]:
df_3.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2021,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,6.897200e+08,113863,74390,...,1106,68698,33363,465,5287,9693,19026,33649,33939,108
1,2021,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,1.217034e+08,52280,29058,...,0,29790,18303,227,3535,5583,11318,15482,11951,35
2,2021,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,9.493560e+06,1221,598,...,187,758,341,0,64,99,198,403,337,12
3,2021,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,1.905703e+07,6307,4191,...,44,2145,3816,24,933,1296,1277,1174,1259,22
4,2021,A,골목상권,2110001,이북5도청사,CS200001,일반교습학원,4.620523e+07,552,544,...,0,296,256,0,0,104,408,24,16,14


In [15]:
case_col = ['남성_매출_건수', '여성_매출_건수',
    '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수',
     '연령대_50_매출_건수', '연령대_60_이상_매출_건수']
rate_col = ['남성_매출_건수_비율', '여성_매출_건수_비율',
    '연령대_10_매출_건수_비율', '연령대_20_매출_건수_비율', '연령대_30_매출_건수_비율', '연령대_40_매출_건수_비율',
     '연령대_50_매출_건수_비율', '연령대_60_이상_매출_건수_비율']

In [16]:
df_3[rate_col] = df_3[case_col].div(df_3['분기당_매출_건수'], axis=0)

In [17]:
df_3[['분기당_매출_건수', '남성_매출_건수', '여성_매출_건수', '남성_매출_건수_비율', '여성_매출_건수_비율']].head()
# 남성매출건수와 여성매출건수의 합이 전체 분기당 매출건수의 합과 일치하지 않음

,분기당_매출_건수,남성_매출_건수,여성_매출_건수,남성_매출_건수_비율,여성_매출_건수_비율
0,113863,68698,33363,0.603339,0.293010
1,52280,29790,18303,0.569816,0.350096
2,1221,758,341,0.620803,0.279279
3,6307,2145,3816,0.340098,0.605042
4,552,296,256,0.536232,0.463768


In [22]:
df_3.to_csv('./data/1.추정매출_전처리완료.csv' , encoding='cp949')